ShuffNet Modeli

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
# Veri Seti 1 ait yol
train_dataset = torchvision.datasets.ImageFolder(root='/content/drive/My Drive/Image_Processing/Veri_Setleri/veri_seti_1/train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = torchvision.datasets.ImageFolder(root='/content/drive/My Drive/Image_Processing/Veri_Setleri/veri_seti_1/Validation', transform=transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

test_dataset = torchvision.datasets.ImageFolder(root='/content/drive/My Drive/Image_Processing/Veri_Setleri/veri_seti_1/test', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)



In [ ]:
# Hard_verisetinin yolu
train_dataset = torchvision.datasets.ImageFolder(root='/content/drive/My Drive/Image_Processing/Veri_Setleri/veri_seti_hard/Train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = torchvision.datasets.ImageFolder(root='/content/drive/My Drive/Image_Processing/Veri_Setleri/veri_seti_hard/Validation', transform=transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

test_dataset = torchvision.datasets.ImageFolder(root='/content/drive/My Drive/Image_Processing/Veri_Setleri/veri_seti_hard/Test', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
model = torchvision.models.shufflenet_v2_x1_0(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # 2 sınıf sahte ve gerçek

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    return epoch_loss

In [ ]:
def validate(model, val_loader, criterion, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    return accuracy

In [ ]:
train_accuracies = []
val_accuracies = []

num_epochs = 50
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    val_accuracy = validate(model, val_loader, criterion, device)
    train_accuracies.append(train_loss)
    val_accuracies.append(val_accuracy)

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}")

In [ ]:
# Test
def test(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    print(f"Test Accuracy: {accuracy:.2f}")

test(model, test_loader, device)

Test Accuracy: 0.92


In [ ]:
import matplotlib.pyplot as plt
'''
# Eğitim ve doğrulama doğruluklarını kaydetmek için boş listeler
train_accuracies = []
val_accuracies = []

# Eğitim ve doğrulama döngüsü
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    val_accuracy = validate(model, val_loader, criterion, device)

    # Doğrulukları listeye ekleme
    train_accuracies.append(val_accuracy)
    val_accuracies.append(val_accuracy)

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}")
'''
test(model, test_loader, device)

plt.plot(range(1, num_epochs+1), train_accuracies, label='Train Loss')
plt.plot(range(1, num_epochs+1), val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation ')
plt.legend()
plt.show()
